In [1]:
import pandas as pd
import numpy as np
import re

# Scammer profiles

In [2]:
scammers_df = pd.read_csv('../data/merged_dataset.csv')
scammers_df.replace("–", np.nan, inplace=True)
scammers_df.head()

,username,name,age,location,ethnicity,occupation,status,phone,inet,email,description,messages,justifications
0,fredJ,Fred Millestone,49,"San Diego, California, United States",white,military,NaN,NaN,178.238.213.161,fred.millestone@yahoo.com,"I am a simple and easy going man,I do not like...",hello beauty can we chat? add me fred.millesto...,IP is a proxy\nIP doesn’t correspond location\...
1,donald200,Donald Steve,46,Ireland,white,Self Employed,widowed,widowed,164.82.146.3,donaldsteve200@yahoo.com,"Am smart, organized, intelligent, honest, cari...","How are you doing,I’m Donald from Ireland… am ...",IP is a proxy\nIP doesn’t correspond location\...
2,huddleman,Peter,45,"Arlington, or Andrews, or Dallas, Texas, Unite...",white,military,widowed,widowed,69.115.19.232,frankiess8928@gmail.com,"I am sweet, thoughtful, kind, intelligent, and...",NaN,IP is a proxy\nIP doesn’t correspond location\...
3,jannycutelove,Jane Douglas,33,"Vancouver, Canada",white,student,single,single,67.191.26.153,janedouglas231@yahoo.com,"I am an attractive, physically and mentally he...",NaN,IP is a proxy\nIP doesn’t correspond location\...
4,Richardholdy,David Holdridge,47,"Bedford, Pennsylvania, United states",white,military,widowed,widowed,199.15.250.244,david_holdy@yahoo.com,"I am a loyal,friendly ,and always supportive ....",Its really a pleasure to hear from back from u...,IP is a proxy\nIP doesn’t correspond location\...


In [3]:
scammers_df = scammers_df.drop(["username", "name", "phone", "inet", "email", "messages", "justifications"], axis=1)

In [4]:
scammers_df = scammers_df.dropna().reset_index(drop=True)

In [5]:
scammers_df

,age,location,ethnicity,occupation,status,description
0,46,Ireland,white,Self Employed,widowed,"Am smart, organized, intelligent, honest, cari..."
1,45,"Arlington, or Andrews, or Dallas, Texas, Unite...",white,military,widowed,"I am sweet, thoughtful, kind, intelligent, and..."
2,33,"Vancouver, Canada",white,student,single,"I am an attractive, physically and mentally he..."
3,47,"Bedford, Pennsylvania, United states",white,military,widowed,"I am a loyal,friendly ,and always supportive ...."
4,32,"Toronto, Canada",white,nurse,single,My goal is a monogamous relationship leading t...
...,...,...,...,...,...,...
3241,49,"Dallas, Texas, United States",native american,engineer,widowed,"I am a straight forward and intelligent man, I..."
3242,59,"Texas City, Texas, United States, or Washingto...",white,military,widowed,Am down to earth and God fearing man who will ...
3243,54 or 60,"Ashburn, Virginia, or Florida, Perry, United S...",white,Architect,widowed,"I am a kind, thoughtful and friendly person. I..."
3244,24,"Hamburg, Germany",black,student,single,"My Name is Jovita, I am a young good looking g..."


In [6]:
statuses = np.asarray(scammers_df["status"].value_counts().index)
idx_to_remove = np.where(np.asarray(scammers_df["status"].value_counts().where(lambda x : x == 1, 0)))[0]
status_to_remove = statuses[idx_to_remove]

In [7]:
status_to_remove

array(['seprated', 'widowed or divorced', 'married or divorced',
       'divorced or single', 'singlescammer’s real name: Olusegun',
       'married (and looking for marriage)', 'Widower', 'military',
       'Single', 'divorce', 'Civil Engineer', 'nurse', 'windoew',
       'Nursing assistant'], dtype=object)

In [8]:
scammers_df = scammers_df[~scammers_df["status"].isin(status_to_remove)]
scammers_df

,age,location,ethnicity,occupation,status,description
0,46,Ireland,white,Self Employed,widowed,"Am smart, organized, intelligent, honest, cari..."
1,45,"Arlington, or Andrews, or Dallas, Texas, Unite...",white,military,widowed,"I am sweet, thoughtful, kind, intelligent, and..."
2,33,"Vancouver, Canada",white,student,single,"I am an attractive, physically and mentally he..."
3,47,"Bedford, Pennsylvania, United states",white,military,widowed,"I am a loyal,friendly ,and always supportive ...."
4,32,"Toronto, Canada",white,nurse,single,My goal is a monogamous relationship leading t...
...,...,...,...,...,...,...
3241,49,"Dallas, Texas, United States",native american,engineer,widowed,"I am a straight forward and intelligent man, I..."
3242,59,"Texas City, Texas, United States, or Washingto...",white,military,widowed,Am down to earth and God fearing man who will ...
3243,54 or 60,"Ashburn, Virginia, or Florida, Perry, United S...",white,Architect,widowed,"I am a kind, thoughtful and friendly person. I..."
3244,24,"Hamburg, Germany",black,student,single,"My Name is Jovita, I am a young good looking g..."


In [9]:
scammers_df = scammers_df[scammers_df["status"] != "single or divorced"]

In [10]:
scammers_df.loc[:, "status"] = scammers_df["status"].apply(lambda x : x.lower())

In [11]:
scammers_df["status"].value_counts()

status
single       1692
widowed       988
divorced      513
separated      22
widower         9
widow           4
married         2
Name: count, dtype: int64

In [12]:
scammers_df.loc[:, "status"] = scammers_df["status"].apply(lambda x : "widowed" if x == "widower" else x)
scammers_df.loc[:, "status"] = scammers_df["status"].apply(lambda x : "widowed" if x == "widow" else x)

# Real profiles

In [22]:
real_df = pd.read_csv('../data/real_profiles.csv')
real_df.replace("-", np.nan, inplace=True)
real_df.head()

,gender,age,location,status,username,ethnicity,occupation,description,match_age,children,orientation,religion,smoking,drinking,intent
0,male,50 y.o.,"East Chicago, IN, USA",widowed,Royer6873,hispanic,NaN,NaN,from 19 to 86,want children,Straight,Other,non-smoker,occasional drinker,Serious Relationship
1,female,42 y.o.,"Mykolaiv, Mykolaiv Oblast, Ukraine, 54000",single,uaola1981,white,NaN,NaN,from 19 to 75,no children,Straight,Other,non-smoker,social drinker,"Friendship, Romance, Serious Relationship, Mar..."
2,female,58 y.o.,"Tulsa, OK, USA",single,Dessarono55,black,Logistics,NaN,from 50 to 60,1-2 living elsewhere,Straight,Spiritual,non-smoker,social drinker,"Serious Relationship, Marriage"
3,male,60 y.o.,"Algiers [El Djazaïr], Algeria",single,maliktabib1964,white,NaN,NaN,from 19 to 50,no children,Straight,Muslim,non-smoker,never,Friendship
4,male,28 y.o.,"Santo Domingo de los Tsáchilas, Ecuador",single,Jaron26,hispanic,Ingeniero Agropecuario,NaN,from 25 to 30,no children,Straight,Christian,non-smoker,never,"Fun, Friendship, Romance, Serious Relationship..."


In [26]:
real_df = real_df.drop(["gender", "username", "children", "orientation", "religion", "smoking", "drinking", "intent", "match_age"], axis=1)

In [27]:
# extract just the age from the age column
real_df["age"] = real_df["age"].str.extract(r'(\d+)')

In [28]:
real_df

,age,location,status,ethnicity,occupation,description
0,50,"East Chicago, IN, USA",widowed,hispanic,NaN,NaN
1,42,"Mykolaiv, Mykolaiv Oblast, Ukraine, 54000",single,white,NaN,NaN
2,58,"Tulsa, OK, USA",single,black,Logistics,NaN
3,60,"Algiers [El Djazaïr], Algeria",single,white,NaN,NaN
4,28,"Santo Domingo de los Tsáchilas, Ecuador",single,hispanic,Ingeniero Agropecuario,NaN
...,...,...,...,...,...,...
5290,30,"San Juan, Puerto Rico",single,hispanic,Fábrica de rolin,NaN
5291,47,"Charlotte, NC, USA",single,hispanic,Demoliccion construcciones,NaN
5292,44,"Bogotá, Bogota, Colombia",single,hispanic,arquitecto,soy una hombre tranquilo y alegre cariñoso dep...
5293,44,"Arenillas, Ecuador",single,hispanic,ingertador de plantas y mas oficios,"soy un hombre sincero honesto trabajador,cariñ..."
